In [ ]:
pip install deepface

##extracting unique faces

In [ ]:
from deepface import DeepFace
import cv2
import os
import shutil
import numpy as np


# Create a directory to store the extracted faces in Google Drive
output_faces_dir = "/content/drive/My Drive/extracted_faces1"
if os.path.exists(output_faces_dir):
    shutil.rmtree(output_faces_dir)
os.makedirs(output_faces_dir)

# Initialize variables
face_db = {}
face_id = 1
num = 9
target_duration_secs = 15
fps = 29

# Load the input video
video_path = f"/content/drive/My Drive/{num}.mp4"
video = cv2.VideoCapture(video_path)

# Check if the video file was opened successfully
if not video.isOpened():
    print("Error reading video file")
    exit()

# Get the total number of frames in the video
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the number of frames to process for the target duration
target_frames = int(target_duration_secs * fps)

# Process the video
processed_frames = 0
while processed_frames < target_frames:
    ret, frame = video.read()

    if ret:
        # Perform face extraction using DeepFace
        faces = DeepFace.extract_faces(frame, detector_backend='retinaface', enforce_detection=False)

        # Extract and store unique faces
        for face in faces:
            x, y, w, h = face["facial_area"]['x'], face["facial_area"]['y'], face["facial_area"]['w'], face["facial_area"]['h']
            extracted_face = frame[y:y+h, x:x+w]

            # Convert the extracted face to grayscale
            gray_extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)

            # Resize the extracted face to a fixed size for better comparison
            resized_extracted_face = cv2.resize(gray_extracted_face, (100, 100))

            # Initialize a flag to check if the extracted face is unique
            unique_face = True

            # Extract embeddings for the extracted face
            extracted_face_embedding = DeepFace.represent(extracted_face, model_name='ArcFace', enforce_detection=False)

            # Compare the extracted face with faces in the database
            for fid, fdata in face_db.items():

                # Extract the embedding from the dictionary
                db_face_embedding_np = np.array(fdata["embedding"])

                # Calculate the cosine similarity between the embeddings
                # similarity = np.dot(extracted_face_embedding[0]["embedding"], db_face_embedding_np[0]["embedding"]) / (np.linalg.norm(extracted_face_embedding[0]["embedding"]) * np.linalg.norm(db_face_embedding_np))
                similarity = DeepFace.verify(extracted_face_embedding[0]["embedding"], db_face_embedding_np[0]["embedding"], model_name='ArcFace', distance_metric='cosine')

                # Calculate the similarity between embeddings
                similarity = similarity['distance']

                if similarity < 0.5:
                  unique_face = False
                  face_id = fid
                  break

            # If the extracted face is unique, assign a new ID to it
            if unique_face:
                # Save the extracted face with a new ID
                face_path = os.path.join(output_faces_dir, f"face_{face_id}.jpg")
                cv2.imwrite(face_path, extracted_face)

                # Update the face database
                face_db[face_id] = {"path": face_path, "embedding": extracted_face_embedding}

                # Increment the face ID
                face_id += 1

        # Increment the number of processed frames
        processed_frames += 1

        # Print progress
        print(f"Progress: {processed_frames}/{target_frames} frames processed")

    else:
        break

# Release video capture
video.release()

# Print the face database
print("\nFace Database:")
for fid, fdata in face_db.items():
    print(f"Face ID: {fid}, Path: {fdata['path']}")

# Save the face database to a file in Google Drive
face_db_path = "/content/drive/My Drive/face_database1.txt"
with open(face_db_path, "w") as f:
    for fid, fdata in face_db.items():
        f.write(f"Face ID: {fid}, Path: {fdata['path']}\n")

print("\nThe unique faces were successfully extracted and saved to:", output_faces_dir)
print("The face database was saved to:", face_db_path)


##Recognizing faces using db

In [ ]:
import cv2
from deepface import DeepFace
import os

# Load the face database from the file
face_db_path = "/content/drive/My Drive/face_database1.txt"
face_db = {}
with open(face_db_path, "r") as f:
    lines = f.readlines()
    for line in lines:
        parts = line.strip().split(", ")
        face_id = int(parts[0].split(": ")[1])
        face_path = parts[1].split(": ")[1]
        face_db[face_id] = face_path

# Load the input video
video_path = "/content/drive/My Drive/9.mp4"
video = cv2.VideoCapture(video_path)

# Check if the video file was opened successfully
if not video.isOpened():
    print("Error reading video file")
    exit()

# Get video properties
fps = int(video.get(cv2.CAP_PROP_FPS))
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Define the codec and create VideoWriter object for output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_path = "/content/drive/My Drive/facerecogtry.mp4"
output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Initialize variables for frame processing
target_duration_secs = 3
target_frames = int(target_duration_secs * fps)
processed_frames = 0

# Process the video frame by frame
while processed_frames < target_frames:
    ret, frame = video.read()

    if not ret:
        break

    # Perform face detection on the frame
    faces = DeepFace.extract_faces(frame, detector_backend='retinaface', enforce_detection=False)

    # Recognize faces and draw bounding boxes
    for face in faces:
        x, y, w, h = face["facial_area"]['x'], face["facial_area"]['y'], face["facial_area"]['w'], face["facial_area"]['h']
        extracted_face = frame[y:y+h, x:x+w]

        # Extract embeddings for the extracted face
        # extracted_face_embedding = DeepFace.represent(extracted_face, model_name='ArcFace', enforce_detection=False)

        # Compare the extracted face with faces in the database
        recognized = False
        for face_id, face_path in face_db.items():
            # Load the reference face
            reference_face = cv2.imread(face_path)

            # Extract embeddings for the reference face
            # reference_face_embedding = DeepFace.represent(reference_face, model_name='ArcFace', enforce_detection=False)

            # Compare the embeddings
            similarity = DeepFace.verify(extracted_face, reference_face, model_name='ArcFace', distance_metric='cosine')
            similarity = similarity['distance']

            # If the similarity is above a threshold, consider the face recognized
            if similarity < 0.5:  # Adjust the threshold as needed
                recognized = True
                break

        # Draw bounding box around the recognized face
        if recognized:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"Face ID: {face_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    output_video.write(frame)
    processed_frames += 1
    print(f"Processed frame {processed_frames}/{target_frames}")

# Release video capture and close windows
video.release()
output_video.release()
cv2.destroyAllWindows()
